In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/04/27 22:09:45 WARN Utils: Your hostname, MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.206.17 instead (on interface en0)
25/04/27 22:09:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/27 22:09:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-22 13:18:32|  2020-01-22 13:45:58|                 N|         1|         244|          41|              1|         5.22|       22.0|  0.0|    0.

# Task1: use RDD to replicate the below group by SQL task

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [5]:
df_green.rdd.take(10)

/usr/local/Cellar/apache-spark/3.5.5/libexec/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=66608) is multi-threaded, use of fork() may lead to deadlocks in the child.


[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), lpep_dropoff_datetime=datetime.datetime(2020, 1, 22, 13, 45, 58), store_and_fwd_flag='N', RatecodeID=1, PULocationID=244, DOLocationID=41, passenger_count=1, trip_distance=5.22, fare_amount=22.0, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=22.8, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 18, 1, 2), store_and_fwd_flag='N', RatecodeID=1, PULocationID=236, DOLocationID=263, passenger_count=1, trip_distance=0.87, fare_amount=6.5, extra=1.0, mta_tax=0.5, tip_amount=1.2, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=12.25, payment_type=1, trip_type=1, congestion_surcharge=2.75),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), lpep_dropo

In [6]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [7]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), PULocationID=236, total_amount=12.25),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), PULocationID=166, total_amount=5.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 14, 25, 16), PULocationID=152, total_amount=17.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 9, 46), PULocationID=51, total_amount=22.01)]

In [9]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [11]:
rows = rdd.take(10)
row = rows[0]

In [12]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8)

In [16]:
from datetime import datetime

In [13]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [14]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [17]:
from collections import namedtuple

In [18]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [19]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [20]:
from pyspark.sql import types

In [21]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [22]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [23]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-22 13:00:00| 244| 589.1100000000001|   28|
|2020-01-19 10:00:00| 166| 969.2699999999993|   55|
|2020-01-26 16:00:00|   7| 682.1099999999998|   50|
|2020-01-27 23:00:00|  65|119.80999999999997|   10|
|2020-01-12 07:00:00| 240|             95.12|    2|
|2020-01-15 09:00:00|  61|            322.29|   11|
|2020-01-04 11:00:00|  39|             78.16|    4|
|2020-01-15 21:00:00|  42| 336.1400000000001|   23|
|2020-01-01 21:00:00|  75| 74.89999999999999|    8|
|2020-01-15 11:00:00|  74| 848.3099999999995|   58|
|2020-01-22 22:00:00|  82| 662.4299999999998|   44|
|2020-01-05 14:00:00| 159|141.88000000000002|    5|
|2020-01-06 08:00:00|  74|1453.3099999999984|  101|
|2020-01-17 09:00:00| 145|175.60000000000002|   19|
|2020-01-26 17:00:00| 130|157.76999999999998|   10|
|2020-01-07 16:00:00|  25| 759.3399999999999|   29|
|2020-01-04 

In [26]:
df_result.write.parquet('tmp/green-revenue')

25/04/27 22:49:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


# Task2: RDD mapPartition

In [27]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [28]:
import pandas as pd

In [29]:
rows = duration_rdd.take(10)

In [30]:
df = pd.DataFrame(rows, columns=columns)

In [31]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [32]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [33]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [34]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [35]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|26.099999999999998|
|              4.35|
|              3.15|
|             13.55|
|10.649999999999999|
|             89.65|
|120.19999999999999|
|              7.25|
|54.699999999999996|
|              15.9|
|               9.3|
|              8.75|
|             94.75|
|              46.2|
|             16.85|
|              14.6|
|             25.65|
|               5.0|
|2.9499999999999997|
|               0.0|
+------------------+
only showing top 20 rows

